In [3]:
#########################################################################################################################
# This notebook works through the construction of the squared exponential kernel, breaking down the mathematics to make #
#                 things as simple as possible. This method uses matrix multiplication to simplify calculations.        #
#########################################################################################################################

import numpy as np

# Generate the input array. Here we have values from 0 to 9, in 5 rows and two columns.
# Let R represent a row. N will be the number of rows, and i and j will represent indices as we iterate through the rows.

p = np.linspace(0, 9, 5).reshape(5, 1)
print p

[[ 0.  ]
 [ 2.25]
 [ 4.5 ]
 [ 6.75]
 [ 9.  ]]


In [4]:
# a is an array with 5 rows and 1 column. Each row contains the sum of the squares of the values in the row.
# b is an array with one row containing the sums of squares. It is the transpose of a.

a = np.sum(p ** 2, 1).reshape(-1, 1)
b = np.sum(p ** 2, 1) 

# So a is a column containing the values of Ri^2, and b is a row containing the values of Ri^2.

print a
print b

[[  0.    ]
 [  5.0625]
 [ 20.25  ]
 [ 45.5625]
 [ 81.    ]]
[  0.       5.0625  20.25    45.5625  81.    ]


In [5]:
# To add the matrices:
# Note this is not adding in the sense of matrix addition - it would not be possible to add a and b as they do not have
# the same shape. What is happening here is numpy 'broadcasting', transforming a and b so that they have the same size. 
# a is repeated 4 times horizontally and b is repeated 4 times underneath the original b. This gives two 5x5 matrices 
# which can be added together.

print a + b

# a + b is a 5x5 matrix. Along the diagonal, each value is the sum of squares of the values in the row, added to itself.
# The off-diagonal values show the sum of squares of the values in one row added to the sum of squares for another row.
# For example, the number in position [4,2] is the sum of squares of the fourth row added to the sum of squares of the
# second row.
# So the matrix values are Ri^2 + Rj^2, and i=j on the diagonal.

[[   0.        5.0625   20.25     45.5625   81.    ]
 [   5.0625   10.125    25.3125   50.625    86.0625]
 [  20.25     25.3125   40.5      65.8125  101.25  ]
 [  45.5625   50.625    65.8125   91.125   126.5625]
 [  81.       86.0625  101.25    126.5625  162.    ]]


In [6]:
# Let's look at the input again, and then look at the transpose of the input.
print p
print p.T

[[ 0.  ]
 [ 2.25]
 [ 4.5 ]
 [ 6.75]
 [ 9.  ]]
[[ 0.    2.25  4.5   6.75  9.  ]]


In [7]:
# Now we will take the dot product of p and its transpose, multiplying each pair of rows together. Take the first row of 
# p and the first column of p.T, multiply the pairs of values (0x0 and 1x1) and add these products together to give 1. 
# To complete the first row of the new matrix, repeat this process with the first row of p and the remaining columns of 
# p.T. Compute the remaining rows by repeating the above with each row of p and the columns of p.T.

d = np.dot(p, p.T) 
print d

# This gives us a 5x5 matrix with the sum of squares of each row along the diagonal. The off-diagonal numbers are the
# result of taking the dot product of different rows. For example, the value in position [5,2], 43, is the result of 
# taking the dot product of the fifth row and the second row ((8x2)+(9x3)).
# Therefore each matrix value is RiRj, and i=j on the diagonal.

[[  0.       0.       0.       0.       0.    ]
 [  0.       5.0625  10.125   15.1875  20.25  ]
 [  0.      10.125   20.25    30.375   40.5   ]
 [  0.      15.1875  30.375   45.5625  60.75  ]
 [  0.      20.25    40.5     60.75    81.    ]]


In [8]:
# Now we will form a matrix with the square of distances between each row. This means that we want (Ri - Rj)^2 (this will
# equal zero if i=j, as we would be computing the distance of a row from itself).
# Multiplying this out, we get Ri^2 + Rj^2 - 2RiRj.

# Recall that the values in a+b represent Ri^2 + Rj^2. We obtain the squared distances by subtracting 2*d, as each value
# in d is RiRj.
#                       (Ri - Rj)^2 = Ri^2 + Rj^2 - 2RiRj
#                                   =  a   +  b -    2*d

sq_dist = a + b - 2*d
print sq_dist

[[  0.       5.0625  20.25    45.5625  81.    ]
 [  5.0625   0.       5.0625  20.25    45.5625]
 [ 20.25     5.0625   0.       5.0625  20.25  ]
 [ 45.5625  20.25     5.0625   0.       5.0625]
 [ 81.      45.5625  20.25     5.0625   0.    ]]


In [9]:
# Our kernel is based on the squared distance. We will multiply the squared distance by a scaling factor and a negative
# number before taking the exponential. This results in a matrix with ones along the diagonal (exp(0)), and positive 
# numbers smaller than one on the off-diagonals.

cov = np.exp(-.5 * (1/0.1) * sq_dist)

# Change the print options to an small number so that each row fits on the screen. This is our covariance matrix.
# A scale factor (the output variance) usually appears before the exponential term.

np.set_printoptions(precision=1)
print cov

[[  1.0e+000   1.0e-011   1.1e-044   1.2e-099   1.3e-176]
 [  1.0e-011   1.0e+000   1.0e-011   1.1e-044   1.2e-099]
 [  1.1e-044   1.0e-011   1.0e+000   1.0e-011   1.1e-044]
 [  1.2e-099   1.1e-044   1.0e-011   1.0e+000   1.0e-011]
 [  1.3e-176   1.2e-099   1.1e-044   1.0e-011   1.0e+000]]
